# Transient Stability and Critical Clearing Time

Automating the calculation of Critical Clearing Time (CCT) for a fault and generating transient response plots to visualize system stability.

In [ ]:
from gridwb import GridWorkBench
from gridwb.grid.components import Branch, Bus
import os

case_path = os.environ.get("SAW_TEST_CASE", "case.pwb")
wb = GridWorkBench(case_path)

In [ ]:
# Initialize Transient Stability
wb.esa.TSInitialize()

# Calculate Critical Clearing Time (CCT) for a branch fault
branches = wb[Branch, ['BusNum', 'BusNum:1', 'LineCircuit']]
if not branches.empty:
    b = branches.iloc[0]
    branch_str = f"[BRANCH {b['BusNum']} {b['BusNum:1']} {b['LineCircuit']}]"
    
    wb.esa.TSCalculateCriticalClearTime(branch_str)
    cct_val = wb.esa.GetParametersSingleElement(
        "Branch", ["TSCritClearTime"], [b['BusNum'], b['BusNum:1'], b['LineCircuit']]
    )
    print(f"Critical Clearing Time: {cct_val:.4f} seconds ({cct_val*60:.1f} cycles)")

In [ ]:
# Generate and save plots
try:
    wb.esa.TSAutoSavePlots(
        plot_names=["Generator Frequencies", "Bus Voltages"],
        contingency_names=["Fault_at_Bus_1"],
        image_file_type="JPG",
        width=1280,
        height=720
    )
    print("Plots saved to the case's result directory.")
except Exception as e:
    print(f"Plot generation skipped: {e}")